<a href="https://colab.research.google.com/github/jensullrich/DataScienceSS20/blob/Project/Project/Project_NLP_disaster_tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Setup


## 1.1 Installation

In [6]:
!pip install tensorflow
!pip install --upgrade tensorflow-hub
!pip install bert-for-tf2
!pip install sentencepiece

Requirement already up-to-date: tensorflow-hub in /usr/local/lib/python3.6/dist-packages (0.8.0)
     |████████████████████████████████| 40kB 3.1MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.4-cp36-none-any.whl size=30114 sha256=43632f98cad21312143122de75608cbc2872c0b734003d336fbffcd92898bfff
  Stored in directory: /root/.cache/pip/wheels/cf/3f/4d/79d7735015a5f523648df90d871ce8e89a7df8185f7703eeab
  Created wheel for py-params: filename=py_params-0.9.7-cp36-none-any.whl size=7302 sha256=313307e2ee38f76d6e27a650ee2ce8b25229f2e18e3c4713c130d3d91a75fb05
  Stored in directory: /root/.cache/pip/wheels/67/f5/19/b461849a50aefdf4bab47c4756596e82ee2118b8278e5a1980
  Created wheel for params-flow: filename=params_flow-0.8.2-cp36-none-any.whl size=19473 sha256=c2ae76563bc2fa0bf15ae5644ac68450607a75688b2e738b179ee02f1b545cad
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params param

## 1.2 Imports

In [7]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub
import bert
from sklearn.preprocessing import LabelEncoder

## 1.3 Cloud

In [9]:
#check if notebook runs in colab
IN_COLAB = 'google.colab' in sys.modules
print('running in Colab:',IN_COLAB)
path='..'
if IN_COLAB:
  #in colab, we need to clone the data from the repo
  !git clone -b Project https://github.com/jensullrich/DataScienceSS20/
  path='DataScienceSS20/Project'

running in Colab: True
Cloning into 'DataScienceSS20'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 739 (delta 16), reused 12 (delta 2), pack-reused 705
Receiving objects: 100% (739/739), 133.40 MiB | 11.20 MiB/s, done.
Resolving deltas: 100% (331/331), done.
Checking out files: 100% (223/223), done.


In [10]:
#check out the GPU
!nvidia-smi

Tue Jun 23 19:19:54 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
#TODO REMOVE
!pip install gputil

import GPUtil as GPU

GPUs = GPU.getGPUs()
gpu = GPUs[0]

def print_GPU_memory():
  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))

print_GPU_memory()

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7413 sha256=ca3f050b91ec78212d8b49160c4ce0e2e5a7ddca7e9ef2afacb1ef369548ae45
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
GPU RAM Free: 11441MB | Used: 0MB | Util   0% | Total 11441MB


# 2. Exploring Data

In [48]:
# read the given csv files
train=pd.read_csv(path + '/DATA/train.csv', index_col='id')
test=pd.read_csv(path + '/DATA/test.csv', index_col='id')
submission = pd.read_csv(path + '/DATA/sample_submission.csv')

In [49]:
train.head()

,keyword,location,text,target
id,,,,
1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


## 2.1 NaN Values

Laut Beschreibung können sowohl keyword, als auch location fehlen. Es wird also zunächst untersucht wie viele Werte fehlen.

In [50]:
training_has_keyword = train["keyword"].notna().value_counts(normalize=True)
training_has_location = train["location"].notna().value_counts(normalize=True)
test_has_keyword = test["keyword"].notna().value_counts(normalize=True)
test_has_location = test["location"].notna().value_counts(normalize=True)

try:
    print(f'{training_has_keyword[True]*100:.1f}', "% of training data have keyword,", f'{(training_has_keyword[False])*100:.1f}', "% of keywords are missing")
    print(f'{training_has_location[True]*100:.1f}', "% of training data have location,", f'{(training_has_location[False])*100:.1f}', "% of locations are missing")
    print(f'{test_has_keyword[True]*100:.1f}', "% of test data have keyword,", f'{(test_has_keyword[False])*100:.1f}', "% of keywords are missing")
    print(f'{test_has_location[True]*100:.1f}', "% of test data have location,", f'{(test_has_location[False])*100:.1f}', "% of locations are missing")
except KeyError:
    print("No NaN values or they have already been replaced")

99.2 % of training data have keyword, 0.8 % of keywords are missing
66.7 % of training data have location, 33.3 % of locations are missing
99.2 % of test data have keyword, 0.8 % of keywords are missing
66.1 % of test data have location, 33.9 % of locations are missing


Die fehlenden Werte bekommen die aussagekräftigeren Einträge 'missing_keyword' bzw. 'missing_location'.

In [51]:
print("Replacing NaN values with missing_keyword and missing_location")
train["keyword"] = train["keyword"].fillna('missing_keyword')
train["location"] = train["location"].fillna('missing_location')
test["keyword"] = test["keyword"].fillna('missing_keyword')
test["location"] = test["location"].fillna('missing_location')

Replacing NaN values with missing_keyword and missing_location


## 2.2 Duplicate Values

In [60]:
duplicates = train[train.duplicated()]
# remove complete duplicates, keep only the first occurence
train.drop_duplicates(keep='first', inplace=True)
# check other duplicates
duplicates = train[train.duplicated(['text'], keep=False)]
duplicates.shape

(106, 4)

## 2.3 Exploration of keyword and location

In [ ]:
print("We have", len(train["keyword"]), "tweets with", train["keyword"].nunique(), "unique keywords.")
print("We have", len(train["location"]), "tweets with", train["location"].nunique(), "unique locations.")

We have 7613 tweets with 222 unique keywords.
We have 7613 tweets with 3342 unique locations.


In [ ]:
grouped = train.groupby(train["keyword"])
grouped.size().sort_values()

grouped = train.groupby(train["location"])
grouped.target.sum().sort_values(ascending=False).head(10)

location
missing_location    1075
USA                   67
United States         27
Nigeria               22
India                 20
Mumbai                19
UK                    16
New York              16
London                16
Washington, DC        15
Name: target, dtype: int64

In [61]:
#encode keyword
labelencoder = LabelEncoder()
train['encoded_keyword'] = labelencoder.fit_transform(train['keyword'])

# 3. Build Model

In [ ]:
training_parameters = {
    "learning_rate": 0.0001,
    "epochs": 10,
    "batch_size": 32
}

In [ ]:
def bert_encode(tweet_texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []

    for text in tweet_texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [ ]:
def build_model(bert_layer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")
    meta_input = Input(shape=(222,), dtype=tf.int32, name='meta_input')

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = concatenate([sequence_output[:, 0, :], meta_input])
    out = Dense(1, activation='sigmoid')(clf_output)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
%%time
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2"
bert_layer = hub.KerasLayer(module_url, trainable=True)

CPU times: user 6.5 s, sys: 722 ms, total: 7.22 s
Wall time: 7.21 s


In [ ]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
FullTokenizer=bert.bert_tokenization.FullTokenizer
tokenizer = FullTokenizer(vocab_file, do_lower_case)

In [ ]:
train_input = bert_encode(train.text.values, tokenizer, max_len=160)
test_input = bert_encode(test.text.values, tokenizer, max_len=160)
train_labels = train.target.values

In [ ]:
model = build_model(bert_layer, max_len=160)
model.summary()

checkpoint = ModelCheckpoint('model.h5', monitor='val_loss', save_best_only=True)

TypeError: ignored

# 4. Training

In [ ]:
train_history = model.fit(
    train_input, train.encoded_keyword, train_labels,
    validation_split=0.2,
    epochs=3,
    callbacks=[checkpoint],
    batch_size=12
)

# 5. Evaluation

In [ ]:
model.load_weights('model.h5')
test_pred = model.predict(test_input)

submission['target'] = test_pred.round().astype(int)
submission.to_csv('submission.csv', index=False)

In [ ]:
printm()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
root_path = '/content/drive/My Drive/INFM/DataScience/Project'

In [ ]:
model.save(root_path + '/models/model_1.h5') 